In [130]:
!pip install pyspark==3.3.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [131]:
from pyspark.sql import SparkSession

In [132]:
spark = SparkSession.builder.master('local[*]').appName("Transformacao com Spark").getOrCreate()

spark

In [133]:
!wget "https://caelum-online-public.s3.amazonaws.com/challenge-spark/semana-1.zip" && unzip semana-1.zip -d dados/

--2022-11-29 17:26:33--  https://caelum-online-public.s3.amazonaws.com/challenge-spark/semana-1.zip
Resolving caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)... 52.216.62.217, 52.216.40.193, 52.216.108.51, ...
Connecting to caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)|52.216.62.217|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18975214 (18M) [application/zip]
Saving to: ‘semana-1.zip.4’

semana-1.zip.4      100%[===================>]  18.10M  46.5MB/s    in 0.4s    

2022-11-29 17:26:34 (46.5 MB/s) - ‘semana-1.zip.4’ saved [18975214/18975214]

Archive:  semana-1.zip
replace dados/dataset_bruto.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: dados/dataset_bruto.json  


In [134]:
dados = spark.read.json('/content/dados/dataset_bruto.json')

In [135]:
dados.show(5, truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------+
|anuncio                                                               

In [136]:
#Verificando a quantidade de registros

dados.count()

89083

In [137]:
dados.printSchema()

root
 |-- anuncio: struct (nullable = true)
 |    |-- andar: long (nullable = true)
 |    |-- area_total: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- area_util: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- banheiros: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- caracteristicas: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- endereco: struct (nullable = true)
 |    |    |-- bairro: string (nullable = true)
 |    |    |-- cep: string (nullable = true)
 |    |    |-- cidade: string (nullable = true)
 |    |    |-- estado: string (nullable = true)
 |    |    |-- latitude: double (nullable = true)
 |    |    |-- longitude: double (nullable = true)
 |    |    |-- pais: string (nullable = true)
 |    |    |-- rua: string (nullable = true)
 |    |    |-- zona: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-

In [138]:
#Filtrando o Data Frame para eliminar as colunas 'imagens' e 'usuario'

anuncio = dados.select('anuncio.*')

anuncio.show(5)

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    0|        []|     [16]|      [0]|                  []|{Centro, 20061003...|47d553e0-79f2-4a4...|    [0]|   [0]|       Usado|      Outros|  Comercial| [1]|[{260, 107, Venda...|
|    0|        []|     [14]|      [0]|                  []|{Centro, 20051040...|b6ffbae1-17f6-487...|    [0]|    []|       Usado|      Outros|  Comercial| [0]|[{260, 107, Venda...|
|    0|    [1026]|   [1026]|      [0]|                  []|{Maria da Graça, ...|1fb030a5-9e3e-4

In [139]:
#Analisando os dados {'tipo_uso':['Comercial', 'Residencial'], 'tipo_anuncio':['Usado', 'Lançamento'], 'tipo_unidade':['Outros', 'Apartamento', 'Casa']}

anuncio.groupBy('tipo_uso').count().show()
anuncio.groupBy('tipo_anuncio').count().show()
anuncio.groupBy('tipo_unidade').count().show()

+-----------+-----+
|   tipo_uso|count|
+-----------+-----+
|  Comercial| 4542|
|Residencial|84541|
+-----------+-----+

+------------+-----+
|tipo_anuncio|count|
+------------+-----+
|       Usado|88827|
|  Lançamento|  256|
+------------+-----+

+------------+-----+
|tipo_unidade|count|
+------------+-----+
|      Outros|11963|
| Apartamento|66801|
|        Casa|10319|
+------------+-----+



In [140]:
#Filtrando o Data Frame para elimenar os dados {'tipo_uso':['Residencial'], 'tipo_anuncio':['Lançamento'], 'tipo_unidade':['Outros', 'Casa']}

anuncio = anuncio.filter("tipo_uso == 'Residencial'")
anuncio = anuncio.filter("tipo_unidade == 'Apartamento'")
anuncio = anuncio.filter("tipo_anuncio == 'Usado'")

anuncio.show()

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    3|      [43]|     [43]|      [1]|[Academia, Churra...|{Paciência, 23585...|d2e3a3aa-09b5-45a...|    [2]|    []|       Usado| Apartamento|Residencial| [1]|[{245, null, Vend...|
|    2|      [42]|     [42]|      [1]|[Churrasqueira, P...|{Paciência, 23585...|085bab2c-87ad-452...|    [2]|    []|       Usado| Apartamento|Residencial| [1]|[{0, 0, Venda, 15...|
|    1|      [41]|     [41]|      [1]|[Portaria 24h, Co...|{Guaratiba, 23036...|18d22cbe-1b86-4

In [141]:
from pyspark.sql import functions as f

In [142]:
#Analisando os a quantidade de elementos das listas registradas nas colunas 'quartos', 'suites', 'banheiros', 'vaga', 'area_total', 'area_util'

caracteristicas = ['quartos', 'suites', 'banheiros', 'vaga', 'area_total', 'area_util']

for i in caracteristicas:
  anuncio.select(f.size(f.col(i)).alias(i)).groupBy(i).count().show()

+-------+-----+
|quartos|count|
+-------+-----+
|      1|66562|
+-------+-----+

+------+-----+
|suites|count|
+------+-----+
|     1|61008|
|     0| 5554|
+------+-----+

+---------+-----+
|banheiros|count|
+---------+-----+
|        1|66562|
+---------+-----+

+----+-----+
|vaga|count|
+----+-----+
|   1|63545|
|   0| 3017|
+----+-----+

+----------+-----+
|area_total|count|
+----------+-----+
|         1|57368|
|         0| 9194|
+----------+-----+

+---------+-----+
|area_util|count|
+---------+-----+
|        1|66562|
+---------+-----+



In [143]:
#Convertendo os registros das colunas 'quartos', 'suites', 'banheiros', 'vaga', 'area_total', 'area_util' de lista para elemento
#OBS: No caso de listas com 1 elemento registra-se o elemento; No caso de listas vazias registra-se null

caracteristicas = ['quartos', 'suites', 'banheiros', 'vaga', 'area_total', 'area_util']

anuncio = anuncio.select([f.col(c)[0].alias(c) if c in caracteristicas else c for c in anuncio.columns])

anuncio.show()

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    3|        43|       43|        1|[Academia, Churra...|{Paciência, 23585...|d2e3a3aa-09b5-45a...|      2|  null|       Usado| Apartamento|Residencial|   1|[{245, null, Vend...|
|    2|        42|       42|        1|[Churrasqueira, P...|{Paciência, 23585...|085bab2c-87ad-452...|      2|  null|       Usado| Apartamento|Residencial|   1|[{0, 0, Venda, 15...|
|    1|        41|       41|        1|[Portaria 24h, Co...|{Guaratiba, 23036...|18d22cbe-1b86-4

In [144]:
#Verifica-se que nem todos os valores de venda estão na primeira célula
valores = anuncio.select('id', 'valores.condominio', 'valores.iptu', 'valores.tipo', 'valores.valor')
valores.show()
valores.groupBy('tipo').count().show()

#Como os valores de condominio e iptu são iguais para venda e aluguel não recebem tratamento
caracteristicas = ['condominio', 'iptu']
valores = valores.select([f.col(c)[0].alias(c) if c in caracteristicas else c for c in valores.columns])
valores.show()

#Tratamento dos dados de valor de compra
from pyspark.sql.functions import when
valores = valores.withColumn("valor", when(valores.tipo[0]=='Venda', valores.valor[0]).when(valores.tipo[1]=='Venda', valores.valor[1]))
valores = valores.withColumn("tipo", when(valores.tipo[0]=='Venda', valores.tipo[0]).when(valores.tipo[1]=='Venda', valores.tipo[1]))
valores.show()
valores.groupBy('tipo').count().show()

+--------------------+----------+------+-------+-------+
|                  id|condominio|  iptu|   tipo|  valor|
+--------------------+----------+------+-------+-------+
|d2e3a3aa-09b5-45a...|     [245]|[null]|[Venda]|[15000]|
|085bab2c-87ad-452...|       [0]|   [0]|[Venda]|[15000]|
|18d22cbe-1b86-476...|       [0]|   [0]|[Venda]|[20000]|
|bed8a354-9317-442...|     [285]|[null]|[Venda]|[20000]|
|12a13315-d67f-48f...|     [245]|[null]|[Venda]|[15000]|
|a2e6d7a5-0ff0-484...|     [285]|[null]|[Venda]|[20000]|
|a6e3173b-c950-4db...|     [250]|[null]|[Venda]|[15000]|
|2e6e5dfb-206c-496...|     [245]|[null]|[Venda]|[15000]|
|99f8d0f9-95a4-461...|     [245]|[null]|[Venda]|[15000]|
|b3f44c1a-2d50-4d5...|     [240]|[null]|[Venda]|[17999]|
|dc99d9e1-4c63-41f...|       [0]|   [0]|[Venda]|[15000]|
|aa3606d8-8bcd-45e...|     [240]|   [0]|[Venda]|[19999]|
|df80b0d5-677c-4be...|     [245]|[null]|[Venda]|[19999]|
|3d6c0218-9b7d-474...|     [290]|[null]|[Venda]|[25000]|
|304266cc-62d1-4ce...|     [285

In [145]:
anuncio = anuncio.join(valores, 'id', how='inner').drop('valores')
anuncio.show()

+--------------------+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+----------+----+-----+------+
|                  id|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|condominio|iptu| tipo| valor|
+--------------------+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+----------+----+-----+------+
|03a386b6-7ab8-4ef...|    0|        43|       43|        1|[Churrasqueira, A...|{Realengo, 217251...|      2|  null|       Usado| Apartamento|Residencial|   1|       285|null|Venda| 22999|
|1fe78d41-b8e0-4d2...|    0|        44|       44|        1|                  []|{Irajá, 21230043,...|      2|     0|       Usado| Apartamento|Residencial|   0|       170|   0|Venda|110000|
|1fa1c1e5-e98c-433...|    4|      null|       55|      